# PlateCarrier

Plate carriers slide into rails on railed-decks like Hamilton STAR(let) and Tecan EVO, and are used to hold Plates.

## Using a plate carrier

The PyLabRobot Resource Library (PLR-RL) has a big number of predefined carriers. You can find these in the [PLR-RL docs](https://docs.pylabrobot.org/resources/index.html). [Hamilton Plate Carriers](https://docs.pylabrobot.org/resources/library/ml_star.html#plate-carriers) may be of particular interest.

In [1]:
from pylabrobot.resources.hamilton import PLT_CAR_L5AC_A00

In [2]:
my_plate_carrier = PLT_CAR_L5AC_A00(name="my_plate_carrier")
my_plate_carrier.capacity

5

To assign a plate at a specific location in the plate carrier, simply set it at a specific index. In PLR, carriers are 0-indexed where the site at the front of the robot (nearest to the door) is 0.

In [3]:
from pylabrobot.resources import Cor_96_wellplate_360ul_Fb

my_plate = Cor_96_wellplate_360ul_Fb(name="my_plate")
my_plate_carrier[0] = my_plate

You can assign plates to a variable and to the carrier in a single line.

In [4]:
my_plate_carrier[1] = my_other_plate = Cor_96_wellplate_360ul_Fb(name="my_other_plate")

The children (in the arborescence) of a plate carrier are {class}`pylabrobot.resources.carrier.PlateHolder` objects. These model the sites for plates on the carrier. A `PlateHolder` may or may not have a `Plate` as a child, depending on whether the spot is occupied.

In [5]:
my_plate_carrier[0]

PlateHolder(name=my_plate_carrier-0, location=Coordinate(004.000, 008.500, 086.150), size_x=127.0, size_y=86.0, size_z=0, category=plate_holder)

In [6]:
my_plate.parent

PlateHolder(name=my_plate_carrier-0, location=Coordinate(004.000, 008.500, 086.150), size_x=127.0, size_y=86.0, size_z=0, category=plate_holder)

You can use the `PlateHolder.resource` attribute to access the `Plate` object, if it exists.

In [7]:
my_plate_carrier[0].resource

Plate(name=my_plate, size_x=127.76, size_y=85.48, size_z=14.2, location=Coordinate(000.000, 000.000, -03.030))

In [8]:
my_plate_carrier[2].resource is None

True

### Moving plates onto carrier sites

If your liquid handling robot has a robotic arm, or if you are using an external robot arm that can interface with carriers, you can move plates out of or onto carriers using the `move_plate` method. For this, you can specify the destination by indexing into the carrier. This will return a `PlateHolder` object.

As an example, we will use the LiquidHandlerChatterboxBackend, but this code will work on any robot that supports moving plates.

In [9]:
from pylabrobot.liquid_handling import LiquidHandler, LiquidHandlerChatterboxBackend
from pylabrobot.resources import STARDeck
lh = LiquidHandler(backend=LiquidHandlerChatterboxBackend(), deck=STARDeck())
lh.deck.assign_child_resource(my_plate_carrier, rails=1)
await lh.setup()

Resource my_plate_carrier was assigned to the liquid handler.
Setting up the liquid handler.
Resource deck was assigned to the liquid handler.
Resource trash was assigned to the liquid handler.
Resource trash_core96 was assigned to the liquid handler.
Resource teaching_carrier was assigned to the liquid handler.
Resource my_plate_carrier was assigned to the liquid handler.


In [10]:
await lh.move_resource(my_plate, my_plate_carrier[2])

Picking up resource: ResourcePickup(resource=Plate(name=my_plate, size_x=127.76, size_y=85.48, size_z=14.2, location=Coordinate(000.000, 000.000, -03.030)), offset=Coordinate(x=0, y=0, z=0), pickup_distance_from_top=0, direction=<GripDirection.FRONT: 1>)
Dropping resource: ResourceDrop(resource=Plate(name=my_plate, size_x=127.76, size_y=85.48, size_z=14.2, location=Coordinate(000.000, 000.000, -03.030)), destination=Coordinate(x=104.0, y=263.5, z=183.12), destination_absolute_rotation=Rotation(x=0, y=0, z=0), offset=Coordinate(x=0, y=0, z=0), pickup_distance_from_top=0, pickup_direction=<GripDirection.FRONT: 1>, drop_direction=<GripDirection.FRONT: 1>, rotation=0)
Resource my_plate was unassigned from the liquid handler.
Resource my_plate was assigned to the liquid handler.


## Pedestal z height

> ValueError("pedestal_size_z must be provided. See https://docs.pylabrobot.org/resources/plate_carriers.html#pedestal_size_z for more information.")

Many plate carriers feature a "pedestal" or "platform" on the sites. Plates can sit on this pedestal, or directly on the bottom of the site. This depends on the pedestal _and_ plate geometry, so it is important that we know the height of the pedestal.

The pedestal information is not typically available in labware databases (like the VENUS or EVOware databases), and so we rely on users to measure and contribute this information.

Here's how you measure the pedestal height:

![Pedestal height measurement](/resources/img/pedestal/measure.jpeg)

Once you have measured the pedestal height, you can contribute this information to the PyLabRobot Labware database. Here's a guide on contributing to the open-source project: ["How to Open Source"](/contributor_guide/how-to-open-source.md).

For background, see PR 143: [https://github.com/PyLabRobot/pylabrobot/pull/143](https://github.com/PyLabRobot/pylabrobot/pull/143).